In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install scikeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 81.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.2 which is incompatible.


In [3]:
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Cropping2D, Input, Rescaling
from tensorflow.keras.applications import ResNet50
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_curve, auc
import pandas as pd
import polars as pl
import numpy as np

2025-10-15 06:02:26.228266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760508146.398015      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760508146.454126      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Things that need to be done in this assignment for colorado: 

1. describe challenge 5pts
2. EDA inspect visualise and clean data 15pts
3. DModel Architecture: dsescribe the model architecture and why I believe this particular one is suitable. Compare multuple architectures and tune hyperparameters (note: use kerastuner for the hyperparameter tuning)
4. results and analysis Include results with tables and figures for why variopus technoquies or architectures worked or did not work well
5. interpret results and describe takeaways

In [4]:
test_dir = '/kaggle/input/histopathologic-cancer-detection/test/'
train_dir = '/kaggle/input/histopathologic-cancer-detection/train/'
train_labels_file = '/kaggle/input/histopathologic-cancer-detection/train_labels.csv'
test_labels_file = '/kaggle/input/histopathologic-cancer-detection/sample_submission.csv'

In [5]:
def load_image(iid, image_dir=train_dir):
    path = image_dir + iid + ".tif"

    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [6]:
train_labels = pl.read_csv(train_labels_file)
n_train_subset = int(train_labels.shape[0] * 0.05)

negative = train_labels.filter(pl.col('label') == 0).sample(n_train_subset)
positive = train_labels.filter(pl.col('label') == 1).sample(n_train_subset)
neg_and_pos = pl.concat([negative, positive])
train_labels_subset = neg_and_pos.sample(fraction=1.0, shuffle=True)

# Extract IDs and create paths
train_image_ids = train_labels_subset['id'].to_list()
train_labels_array = train_labels_subset['label'].to_numpy()

train_dir = '/kaggle/input/histopathologic-cancer-detection/train/'
train_image_paths = [f"{train_dir}/{img_id}.tif" for img_id in train_image_ids]

# Use tf.data for parallel loading
def load_and_preprocess(path, label):
    image = tf.io.read_file(path)
    image = tf.io.decode_image(image, channels=3, expand_animations=False)
    image.set_shape([96, 96, 3])
    image = tf.cast(image, tf.float32) / 255.0
    # Add GPU-friendly augmentations
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.1)
    return image, label

# Create dataset
dataset = tf.data.Dataset.from_tensor_slices((train_image_paths, train_labels_array))
dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)

I0000 00:00:1760508158.655956      37 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Preprocessing we are varying brigthness, orientation intesnity and saturation to avoid overfitting. 

In [7]:
import tensorflow as tf
import polars as pl
import os

# === Keep Polars Sampling Logic ===
train_labels = pl.read_csv(train_labels_file)
test_labels = pl.read_csv(test_labels_file)

n_train_subset = int(train_labels.shape[0] * 0.05)
negative = train_labels.filter(pl.col('label') == 0).sample(n_train_subset)
positive = train_labels.filter(pl.col('label') == 1).sample(n_train_subset)
neg_and_pos = pl.concat([negative, positive])
train_labels_subset = neg_and_pos.sample(fraction=1.0, shuffle=True)

# Extract IDs and labels
train_image_ids = train_labels_subset['id'].to_list()
train_labels_array = train_labels_subset['label'].to_numpy()

# Base path
train_dir = '/kaggle/input/histopathologic-cancer-detection/train/'

# 🔍 Auto-detect file extension (.png vs .tif)
sample_file_png = os.path.join(train_dir, train_image_ids[0] + '.png')
use_png = os.path.exists(sample_file_png)
ext = 'png' if use_png else 'tif'

# Build full paths
train_image_paths = [os.path.join(train_dir, f"{img_id}.{ext}") for img_id in train_image_ids]

# === TensorFlow Preprocessing Pipeline ===
def load_and_preprocess_tf(path, label):
    # Read and decode image
    image = tf.io.read_file(path)
    image = tf.io.decode_image(image, channels=3, expand_animations=False)
    image.set_shape([96, 96, 3])  # Ensure static shape
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]

    # Apply GPU-accelerated augmentations
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, 0.1)
    image = tf.image.random_contrast(image, 0.9, 1.1)
    image = tf.image.random_saturation(image, 0.9, 1.1)

    return image, label

# Create dataset
dataset = tf.data.Dataset.from_tensor_slices((train_image_paths, train_labels_array))

# Shuffle for better training dynamics
dataset = dataset.shuffle(buffer_size=1000, seed=42)

# Map preprocessing with parallelization
dataset = dataset.map(load_and_preprocess_tf, num_parallel_calls=tf.data.AUTOTUNE)

# Batch and prefetch for performance
dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)




It is a standard CNN. I did add some batch normalization layers in order to stabilize and improve training. 

In [8]:
def build_model(hp):
    model = Sequential([
        Input(shape=(96, 96, 3)),
        Rescaling(1./255),
        Cropping2D(cropping=32),
            
        Conv2D(32, (3,3), activation='relu'),
        #BatchNormalization(),
        MaxPooling2D(2,2),
            
        Conv2D(hp.Int("conv_units", 64, 128, step=64), (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),
            
        Flatten(),
        Dense(hp.Int("dense_units", 128, 256, step=128), activation='relu'),
        Dropout(0.3),
        BatchNormalization(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(hp.Choice("learning_rate", [1e-4, 1e-3])),
                  loss='binary_crossentropy', metrics=['accuracy', 'auc'])
    return model

In [9]:
# model.compile(optimizer=Adam(hp.Choice("learning_rate", [1e-4, 1e-3])),
#                   loss='binary_crossentropy', metrics=['accuracy', 'auc'])

In [10]:
from tensorflow.keras.utils import load_img, img_to_array
import tensorflow as tf
import numpy as np


In [11]:
import keras_tuner
import keras

In [21]:
# X_train, X_val, y_train, y_val = train_test_split(
#     X, 
#     y, 
#     test_size=0.2, 
#     random_state=42, 
#     stratify=y  # Ensures balanced class distribution
# )
import tensorflow as tf
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split

desired_height = 96
desired_width = 96

def load_tif_image(path):
    path = path.numpy().decode("utf-8")
    img = Image.open(path)
    arr = np.array(img)
    return arr

def load_and_preprocess_tf(path, label):
    image = tf.py_function(load_tif_image, [path], tf.uint8)
    image.set_shape([None, None, None])
    image = tf.image.resize(image, [desired_height, desired_width])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Split between training and validation sets
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_image_paths,
    train_labels_array,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# Build TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_dataset = train_dataset.map(load_and_preprocess_tf, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(1000, seed=42).batch(32).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_dataset = val_dataset.map(load_and_preprocess_tf, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)




# tuner = keras_tuner.RandomSearch(
#     build_model,
#     objective='val_loss',
#     max_trials=5,
#     directory='my_dir',
#     project_name='my_project',
#     overwrite=True
# )
# #tuner.search(X, y, epochs=5)#, validation_data=(X_val, y_val))

# # Pass the newly created validation set to the search method
# #tuner.search(X_train, y_train, epochs=5, validation_data=(X_val, y_val))
# tuner.search(train_dataset, epochs=2, validation_data=(val_dataset))
# best_hp = tuner.get_best_hyperparameters(1)[0]
# print(best_hp)

In [22]:
tuner = keras_tuner.Hyperband(
    hypermodel=build_model,
    objective="val_accuracy",
    max_epochs=2,
    factor=3,
    hyperband_iterations=1,
    distribution_strategy=tf.distribute.MirroredStrategy(),
    directory="results_dir",
    #project_name="mnist",
    #overwrite=True,
)

In [23]:

tuner.search(train_dataset,validation_data=(val_dataset), callbacks=[keras.callbacks.EarlyStopping("val_accuracy")] )


Trial 2 Complete [00h 01m 15s]
val_accuracy: 0.7343785762786865

Best val_accuracy So Far: 0.7343785762786865
Total elapsed time: 00h 02m 54s


In [24]:
best_hp = tuner.get_best_hyperparameters(1)[0]

print(best_hp)

In [25]:
import os
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array

test_dir = '/kaggle/input/histopathologic-cancer-detection/test/'

# List only .tif, .png files
valid_extensions = {'.tif', '.tiff', '.png'}
test_image_paths = [
    os.path.join(test_dir, fname) 
    for fname in sorted(os.listdir(test_dir)) 
    if os.path.splitext(fname.lower())[1] in valid_extensions
]

print(f"Found {len(test_image_paths)} image files.")

def load_and_preprocess_with_keras(path):
    try:
        # Convert Tensor to string
        path_str = path.numpy().decode('utf-8')
        # Load image with Keras (uses Pillow, supports all formats)
        img = load_img(path_str, target_size=(96, 96), color_mode='rgb')
        image = img_to_array(img)
        image = tf.cast(image, tf.float32) / 255.0
        return image
    except Exception as e:
        tf.print(f"Error loading {path_str}: {e}")
        # Return blank image to avoid breaking the batch
        return tf.zeros((96, 96, 3), dtype=tf.float32)

# Use tf.py_function to wrap Python logic
test_dataset = tf.data.Dataset.from_tensor_slices(test_image_paths)
test_dataset = test_dataset.map(
    lambda path: tf.py_function(load_and_preprocess_with_keras, [path], tf.float32),
    num_parallel_calls=tf.data.AUTOTUNE
)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)






Found 57458 image files.


In [26]:
best_model = tuner.hypermodel.build(best_hp)
predictions = best_model.predict(test_dataset)

# Convert probabilities to binary labels (0 or 1)
# Or keep as probabilities if challenge expects them
predicted_labels = (predictions > 0.5).astype(int).flatten()

1796/1796 ━━━━━━━━━━━━━━━━━━━━ 159s 88ms/step


In [27]:
print(predicted_labels)

[1 1 1 ... 1 1 1]


In [29]:
import polars as pl

# If you need the correct 'id', uncomment, but ensure path/file exists
sample_sub = pl.read_csv('/kaggle/input/histopathologic-cancer-detection/sample_submission.csv')

# Create submission DataFrame
submission = pl.DataFrame({
    'id': sample_sub['id'],  # If you have IDs from sample_sub
    'label': predicted_labels  # Or y_pred_reshaped.flatten() or your predictions array
})

# Save to CSV in your current working directory
submission.write_csv('submission.csv')

conclusion: I applied a CNN and added batch normilzation to increase accuracy and stabilize training. I applied brigthness, orientation intesnity and saturation to avoid overfitting. 